# Detector de língua

## Objetivo

Elaborar uma estratégia e construa um sistema que seja capaz de responder a qual idioma uma frase qualquer pertence.

Por Exemplo:

`eu gosto de café` → `por`

`I like coffee`→ `eng`

## Bibliotecas

In [1]:
# manipulação de dados
import pandas as pd

# manipulação de texto

## Base de Dados

Dessa vez encontrei uma base de dados contendo frases em seus respectivos idiomas Afrikaans, English e Nederlands (africano, inglês e holandês). Queria achar alguma em português mas não consegui.

Disponível no seguinte repositório do github: https://github.com/rolandgem/LanguageClassification/tree/main

## Leitura dos dados 

In [4]:
df = pd.read_csv('lang_data.csv')
df.head()

,text,language
0,Ship shape and Bristol fashion,English
1,Know the ropes,English
2,Graveyard shift,English
3,Milk of human kindness,English
4,Touch with a barge-pole - Wouldn't,English


In [5]:
df.shape

(2839, 2)

In [8]:
df.duplicated().sum()

84

In [16]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [17]:
df.isnull().sum()

text        3
language    0
dtype: int64

In [18]:
df.dropna(inplace=True)
df.isnull().sum()

C:\Users\Ben-Hur\AppData\Local\Temp\ipykernel_8924\3641017954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


text        0
language    0
dtype: int64

In [19]:
df.shape

(2752, 2)

In [20]:
percentual_lingua = df['language'].value_counts(normalize=True) * 100
percentual_lingua


English       74.382267
Afrikaans     23.183140
Nederlands     2.434593
Name: language, dtype: float64

**Temos muito pouca amostra de textos em holandês**

## Preparação dos textos

### Stop Words

Aqui teremos que separar as 3 línguas e usar pacotes para remover as stop words dos 3 idiomas distintos.

In [21]:
df1 = df.copy()

df_africano = df1[df1['language'] == 'Afrikaans']
df_ingles = df1[df1['language'] == 'English']
df_holandes = df1[df1['language'] == 'Nederlands']